In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
def normalize(x, negative=False, inplace=False):
    if inplace == True:
        for i in range(x.shape[0]):
            for j in range(x.shape[1]):
                if negative == True:
                    x[i,j] = 2*x[i,j] / 255 -1
                else:
                    x[i,j] = x[i,j]/255
        return x
    else:
        newarray = np.ndarray(x.shape)
        for i in range(x.shape[0]):
            for j in range(x.shape[1]):
                if negative == True:
                    newarray[i,j] = 2*x[i,j] / 255 -1
                else:
                    newarray[i,j] = x[i,j]/ 255
        return newarray

In [5]:
fulldata = np.load("fashion_test.npy")
data = fulldata[:,:-1]
normal_test = normalize(data)
np.save('normalized_test',normal_test)
normal_test

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.02745098, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.13333333, 0.        ,
        0.        ]])

In [5]:
def windowfilter(x, kernel, imgdim=28):
    margin = int(np.floor(kernel.shape[0] / 2))
    retarray = np.ndarray(x.shape)
    for img in range(x.shape[0]):
        tempimg = np.reshape(x[img], (imgdim, imgdim))
        rescoredimg = np.zeros((imgdim, imgdim))
        #copy original border pixels
        for i in range(-margin, margin):
            rescoredimg[i] = tempimg[i]
            rescoredimg.T[i] = tempimg.T[i]
        for ridx in range(margin, imgdim-margin):
            for cidx in range(margin, imgdim-margin):
                local = []
                for i, kr in enumerate(range(-margin, margin+1)):
                    for j, kc in enumerate(range(-margin, margin+1)):
                        local.append(tempimg[ridx+kr,cidx+kr]*kernel[i,j])
                #print(sum(local))
                rescoredimg[ridx,cidx] = sum(local)/np.sum(kernel)
        retarray[img] = rescoredimg.flatten()
    return retarray
    
                

In [ ]:
gk3 = np.array([[1, 2, 1], [2, 4, 2], [1, 2, 1]])
gk5 = np.array([[1, 4, 6, 4, 1], [4, 16, 24, 16, 4], [6, 24, 36, 24, 6], \
               [4, 16, 24, 16, 4], [1, 4, 6, 4, 1]])

In [157]:
lowpassed = windowfilter(data,gk5)

In [100]:
#https://github.com/jmlipman/LAID/blob/master/IP/Otsu/otsu.py
def otsu(x):
    thrlist = []
    for i in x:
        [hist, _] = np.histogram(i, bins=256, range=(0, 1))
        hist = 1.0*hist/np.sum(hist)
        val_max = -999
        thr = -1
        for t in range(1,255):
            # Non-efficient implementation
            q1 = np.sum(hist[:t])
            q2 = np.sum(hist[t:])
            m1 = np.sum(np.array([i for i in range(t)])*hist[:t])/q1
            m2 = np.sum(np.array([i for i in range(t,256)])*hist[t:])/q2
            val = q1*(1-q1)*np.power(m1-m2,2)
            if val_max < val:
                val_max = val
                thr = t
        thrlist.append(thr/255)
    return thrlist

In [154]:
thr = otsu(lowpassed)

/usr/lib/python3/dist-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in double_scalars
  


In [131]:
def thresholding(x, thrlist):
    out = np.ndarray(x.shape)
    for row in range(x.shape[0]):
        for col in range(x.shape[1]):
            if  x[row,col] <= thrlist[row]:
                out[row,col] = 0
            else:
                out[row,col] = 1
    return out

In [155]:
trashed = thresholding(lowpassed, thr)

In [160]:
#if you wanna look at an image after thresholding
from PIL import Image

im = np.reshape(trashed[2], (28, 28))
img = Image.fromarray(im)
img = img.convert("L")
img.save("thrtest2.png")

In [156]:
np.save("gauss5_binary01.npy", trashed)